In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models.phrases import Phrases, Phraser
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import yake
from langdetect import detect, detect_langs, DetectorFactory
import re

In [2]:
# read csv
authors = pd.read_csv('top_20_authors.csv')
publications = pd.read_csv('publications-top_20_authors.csv', sep=',')

In [7]:
# make a dictionary containing the combined abstracts for each author
author_publication_pairs = list(zip(publications['user_id'], publications['abstract_text']))

DetectorFactory.seed = 0

authors_texts = {author_id:"" for author_id in authors["id"]}
for author_id, abstract in author_publication_pairs:
    if abstract and isinstance(abstract, str) and re.match('^(?=.*[a-zA-Z])', abstract):
        language = detect(abstract)
        if language != 'en':
            print(language)
        else:
            authors_texts[author_id] = authors_texts[author_id] + "\n" + abstract

ro
ro
ro
it
it
ro
ro
id
id
ro
de
ro
ro
ro
ro
ro
ro
ro
ro
pt
ro
ro
ro
ro
ro
ro
fr
fr
ro
ro
ro
ro
ro
ro
ro
ro
fr
fr
fr
ro
ro
ro
fr
ro
ro
ro
ro
fr
ro
fr
fr
ro
ro
ro
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
ro
ro
de
de
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
ro
ro
ja
ro
ro
fr
es
ro
ro
ro
ro
de
ro
ro
ro
ro
ro
ro
fr
ro
et
ro
ro
ro
et
ro
fr
ro
fr
fr
fr
fr
de
ro
ro
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
pt
pt
ro
ro
fr
et
ro
ro
ro
tr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
de
de
ro
ro
de
de
de
de
de
de
ro
de
ro
ro
ro
ca
ro
ro
ro
ro
fr
es
fr
ro
de
de
so
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
fr
fr
fr
fr
de
de
de
de
de
fr
ro
ro
ro
de
fr
ro
ro
ro
ro
ca
ro
ro
ro
ro
ro
ro
ro
fr
fr
ro
ro
fr
ro
ro
ro
ro
it
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
de
fr


In [8]:
nlp = spacy.load('en_core_web_md')

EXTRACT KEYWORDS/KEYPHRASES WITH YAKE

In [13]:
kw_extractor = yake.KeywordExtractor()
max_ngram = 2
deduplication_threshold = 0.2
keywords_nr = 5
custom_kw_extractor = yake.KeywordExtractor(lan="en", n=max_ngram, dedupLim=deduplication_threshold, top=keywords_nr, features=None)

text = authors_texts[1146]
doc = nlp(text)
remove_entities = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY',\
                   'QUANTITY', 'CARDINAL', 'ORDINAL']
stop_words = ['paper', 'present', 'propose', 'datum', 'people', 'result', 'solution', 'case', 'order', 'base', 'ieee', 'privacy', 'policy',
             'new', 'old', 'context', 'high', 'different', 'new', 'old', 'research', 'type', 'approach', 'important', 'main', 'range',
             'helpful', 'large', 'difficult', 'available', 'amount', 'useful', 'importance', 'article', 'abstract', 'scale', 'copyright',
             'real', 'quality', 'inconvenience', 'benefit', 'unavailable', 'term', 'condition', 'interest', 'organization', 'use',
             'task', 'student', 'professor', 'teacher', 'university']

transformed_text = ' '.join([token.text for token in doc if token.is_alpha and token.ent_type_ not in remove_entities \
                             and token.lemma_.lower() not in stop_words and token.pos_ != 'ADV'])

keywords = custom_kw_extractor.extract_keywords(transformed_text)
for kw in keywords:
    print(kw)

('virtual reality', 6.566062282347345e-07)
('Online virtual', 7.007194096160696e-06)
('learning process', 1.3801336482879942e-05)
('game learning', 3.0353921310313012e-05)
('computer games', 3.076373142927375e-05)


EXTRACT TOPICS WITH GENSIM LDA

In [14]:
# make a dictionary containing a list of abstracts for each author
author_publication_pairs = list(zip(publications['user_id'], publications['abstract_text']))

authors_texts = {author_id:[] for author_id in authors["id"]}
for author_id, abstract in author_publication_pairs:
    if abstract and isinstance(abstract, str) and re.match('^(?=.*[a-zA-Z])', abstract):
        language = detect(abstract)
        if language == 'en':
            authors_texts[author_id].append(abstract)

In [20]:
texts = authors_texts[534]

remove_pos = ['ADV', 'PRON', 'PART', 'DET', 'SPACE', 'NUM', 'SYM', 'ADP', 'VERB', 'CCONJ']
stop_words = ['paper', 'present', 'propose', 'datum', 'people', 'result', 'solution', 'case', 'order', 'base', 'ieee', 'privacy', 'policy',
             'new', 'old', 'context', 'high', 'different', 'new', 'old', 'research', 'type', 'approach', 'important', 'main', 'range',
             'helpful', 'large', 'difficult', 'available', 'amount', 'useful', 'importance', 'article', 'abstract', 'scale', 'copyright',
             'real', 'quality', 'inconvenience', 'benefit', 'unavailable', 'term', 'condition', 'interest', 'organization', 'use',
             'task', 'student', 'professor', 'teacher', 'university']

# keep only adjectives and nouns
remove_entities = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY',\
                   'QUANTITY', 'CARDINAL', 'ORDINAL']

tokens = []
for abstract in texts:
    doc = nlp(abstract)
    t = [token.lemma_.lower() for token in doc if token.is_alpha and token.ent_type_ not in remove_entities \
                             and token.lemma_.lower() not in stop_words and token.pos_ not in remove_pos and not token.is_stop]
    tokens.append(t)

# add bigrams to the token list
bigram = Phrases(tokens, min_count=2, delimiter=' ', threshold=1)
tokens = [bigram[text] for text in tokens]
trigram = Phrases(bigram[tokens], min_count=2, delimiter=' ', threshold=1)
tokens = [bigram[text] for text in tokens]

In [21]:
# remove single words
tokens = [[token for token in text if len(token.split(" ")) > 1] for text in tokens]

In [22]:
print(tokens)

[['city smart'], ['monalisa monitoring', 'agents integrated', 'services architecture', 'service architecture', 'grid system', 'data intensive', 'scalability system', 'dynamic service', 'service client', 'network performance', 'agent framework', 'global optimization', 'application site'], ['data intensive', 'application area', 'experiment simulation', 'internet technology', 'everyday object'], ['monalisa monitoring', 'agents integrated', 'services architecture', 'set service', 'monitoring control', 'management global', 'multi agent', 'dynamic service', 'service client', 'control global', 'time information'], ['cloud computing', 'ad hoc', 'parallel processing', 'parallel processing', 'major challenge'], ['late generation', 'monarc models', 'networked analysis', 'regional centers', 'simulation framework', 'tool system', 'hep experiment', 'discrete event', 'running program', 'pattern system', 'behavior processing', 'appropriate scheduling', 'way complex', 'running program', 'shared resourc

In [23]:
# create dictionary with gensim
dictionary = Dictionary(tokens)
# create corpus
corpus = [dictionary.doc2bow(text) for text in tokens]

In [24]:
# apply LDA
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=500, num_topics=1, workers=4, passes=200)
topics = lda_model.print_topics(-1)

for idx, topic in topics:
    print(f"{idx}: {topic}")

0: 0.012*"mobile device" + 0.009*"opportunistic network" + 0.005*"cloud computing" + 0.004*"service updated" + 0.004*"information product" + 0.004*"fault tolerance" + 0.004*"account management" + 0.004*"technology humanity" + 0.004*"single sale" + 0.004*"site agreement"


EXTRACT KEYWORDS FROM PUBLICATION TITLES

In [305]:
# make a dictionary containing the combined titles for each author
author_title_pairs = list(zip(publications['user_id'], publications['title']))

authors_titles = {author_id:"" for author_id in authors["id"]}
for author_id, title in author_title_pairs:
    if title and isinstance(title, str):
        authors_titles[author_id] += "\n" + title

In [306]:
kw_extractor = yake.KeywordExtractor()
max_ngram = 2
deduplication_threshold = 0.2
keywords_nr = 3
custom_kw_extractor = yake.KeywordExtractor(lan="en", n=max_ngram, dedupLim=deduplication_threshold, top=keywords_nr, features=None)

text = authors_titles[562]
doc = nlp(text)

transformed_text = ' '.join([token.text for token in doc if token.is_alpha])

keywords = custom_kw_extractor.extract_keywords(transformed_text)
for kw in keywords:
    print(kw)

('Cloud Computing', 3.0867732827543088e-06)
('Big Data', 3.115603220655388e-06)
('distributed systems', 3.5027023281787467e-06)
